# Natural Language to SQL Generation Project

**Subject:** DBMS (Assignment)  
**Submitted By:** JAGANNATH BHARALI  
**Roll Number:** 230710007017

***

## 1. Project Overview

This project demonstrates a sophisticated system for converting natural language questions into executable SQL queries. The core functionality allows a user to interact with a database using plain English, which the system translates into accurate, syntactically correct SQL.

The system successfully fulfills all **V1 to V10 requirements** by implementing an advanced **ML-Based Seq2Seq architecture (V13)**. This demonstrates its capability to handle a wide spectrum of query complexities, including:

* Basic **single-table queries** (V1) and **multi-condition filtering** using `AND`/`OR` (V2).
* Core SQL operations including **aggregations** like `COUNT` and `AVG` (V3), **sorting** with `ORDER BY` (V4), and **grouping** with `GROUP BY` (V5).
* Handling of **multi-table queries** through `JOIN` operations (V6).
* Robustness in understanding user intent through **synonym handling** (V7) and processing specific data types like **time/date queries** (V9).
* Generation of advanced logic, including **comparative** (V8) and **nested** (V10) queries using subqueries.

---

## 2. Methodology and Project Evolution

The development of this project involved a significant evolution in approach, reflecting a practical journey in solving modern AI and data access challenges.

### Initial Approach (Rule-Based & Hosted Models)
The initial plan was to use regular expressions or a simple hosted model from a platform like Hugging Face. However, this approach faced two major technical hurdles:
1.  **Model Availability:** Many community-hosted models were found to be unstable, frequently becoming unavailable or being removed, which led to persistent `RepositoryNotFound` errors.
2.  **Environmental Instability:** The Google Colab environment consistently failed to connect to the Hugging Face Hub for model downloads. Switching to the Hugging Face Inference API also proved unreliable, with API endpoints for required models often being unavailable (`404 Not Found` errors).

### The Final, Robust Solution: Google's Gemini API
Given the instability of external dependencies, the project was pivoted to a more robust and integrated solution: **Google's Gemini API**. This approach was chosen for several key reasons:
* **Reliability:** As we are working in a Google Colab environment, using Google's native AI service guarantees stable and reliable network access, eliminating all previous connection issues.
* **State-of-the-Art Performance:** The **Gemini 1.5 Flash** model is a powerful, instruction-tuned LLM that excels at reasoning and code generation tasks. As demonstrated by the successful outputs, it is ideal for high-accuracy Text-to-SQL translation.
* **Contextual Understanding:** The model's effectiveness is significantly enhanced by providing the database schema in the form of `CREATE TABLE` statements. This "prompt engineering" technique gives the AI the precise context it needs to generate accurate queries that match the specific table and column names.

---

## 3. How It Works

The final system operates on a simple yet powerful principle:

1.  **Schema Definition:** The structure of the database is defined using standard `CREATE TABLE` statements. This context is fed to the AI.
2.  **Prompt Engineering:** A detailed prompt is constructed for the AI, containing the user's natural language question and the complete database schema.
3.  **API Call:** This structured prompt is sent to the Gemini API.
4.  **SQL Generation:** The Gemini model analyzes the request and the provided schema to generate a corresponding, syntactically correct SQL query.
5.  **Output:** The generated SQL query is displayed to the user.

---

## 4. How to Use This Notebook

Here’s a simple guide to run this project:

### **1. API Key Information (Important)**

**For convenience, a Google AI API key has already been saved in this notebook's Secrets Manager.** You should be able to run all the cells directly without any setup.

*If you prefer to use your own free API key, please follow the optional steps below. Adding your own key with the name `GOOGLE_API_KEY` will override the existing one for your session.*

#### **Optional: To Use Your Own API Key**

* **Get the Key:**
    1.  Go to **[Google AI Studio](https://makersuite.google.com/app/apikey)**.
    2.  Click **`Create API key in new project`**.
    3.  Copy the new key that is generated.

* **Add the Key to Colab:**
    1.  In this notebook, click the **key icon (🔑)** on the left sidebar to open the "Secrets" panel.
    2.  Click **`+ Add a new secret`**.
    3.  Enter the details exactly as follows:
        * **Name:** `GOOGLE_API_KEY`
        * **Value:** Paste the key you copied.
    4.  Make sure the **'Notebook access'** toggle is switched on.

### **2. Run the Project Cells in Order**
Execute the primary project cells sequentially from top to bottom. This action will:
* Install all necessary libraries.
* Configure the connection to the Gemini API using the provided key.
* Define the database schema and run all pre-defined test cases to validate that the V1-V10 requirements are met.

### **3. Use the Interactive Custom Schema Tester**
Run the final cell in the notebook. This powerful tool allows you to:
* **Input any database schema** using `CREATE TABLE` statements.
* Once the schema is entered, you can **ask questions in natural language** about your custom tables.
* This demonstrates the system's flexibility and its ability to adapt to any database structure on the fly.


# **Main Project Cell-1**

In [ ]:

# --- 1. Install the Google AI Python SDK ---
!pip install -q google-generativeai
print("✅ Google AI library installed.")

# --- 2. Imports and API Configuration ---
import google.generativeai as genai
from google.colab import userdata
import textwrap
import time

try:
    # Get your API key from Colab Secrets
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Google AI API configured successfully.")
except Exception as e:
    print("🛑 API Key not found. Please follow the steps to add your GOOGLE_API_KEY to Colab Secrets.")

# --- 3. The Gemini-based SQL Generation Engine ---
# Use the new, correct model name: 'gemini-1.5-flash'
try:
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("✅ Gemini 1.5 Flash model loaded successfully.")
except Exception as e:
    print(f"🛑 Error loading model: {e}")
    model = None


def generate_sql_with_gemini(natural_language_query, create_table_statements):
    """
    Generates an SQL query using the Gemini model.
    """
    if not model:
        return "Model not loaded. Cannot generate SQL."

    # Create a very clear, structured prompt for the model
    prompt = textwrap.dedent(f"""
    Given the following SQL tables, your job is to write queries given a user's request.

    ### CREATE TABLE statements:
    {create_table_statements}

    ### User's question:
    {natural_language_query}

    ### SQL Query:
    """).strip()

    try:
        # Call the model to generate the content
        response = model.generate_content(prompt)
        # Clean up the response to get only the SQL query
        sql_query = response.text.strip().replace("```sql", "").replace("```", "").strip()
        return sql_query
    except Exception as e:
        # A delay and retry can help if the service is temporarily busy
        print(f"    An error occurred. Retrying in 5 seconds... Error: {e}")
        time.sleep(5)
        try:
            response = model.generate_content(prompt)
            sql_query = response.text.strip().replace("```sql", "").replace("```", "").strip()
            return f"[RETRY SUCCESS] {sql_query}"
        except Exception as retry_e:
            return f"An error occurred with the Gemini API on retry: {retry_e}"

print("✅ Gemini SQL Generation Engine is ready.")


✅ Google AI library installed.
✅ Google AI API configured successfully.
✅ Gemini 1.5 Flash model loaded successfully.
✅ Gemini SQL Generation Engine is ready.


# **Main Project Cell-2**

In [ ]:

# --- 4. Define Schema and Generate All Queries ---
schema_statements = """
CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    grade TEXT,
    marks INTEGER,
    date_of_admission DATE
);
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL,
    student_id INTEGER,
    FOREIGN KEY (student_id) REFERENCES students (id)
);
"""
print("\n--- Database Schema Context ---")
print(schema_statements)

queries_to_test = [
    # V1: Basic Rule-Based Query (Simple WHERE)
    "Show all students with marks > 80",

    # V2: Multi-Condition Query (WHERE with AND)
    "List students older than 20 and with grade 'A'",

    # V3: Aggregation Query (AVG)
    "Find the average marks of students",

    # V4: Sorting Query (ORDER BY)
    "Show students ordered by marks descending",

    # V5: Grouping Query (GROUP BY)
    "Show average marks by grade",

    # V6: Multi-Table Query (JOIN)
    "Show student names with their course names",

    # V7: Synonym & Phrasing Variation
    "What are the scores for students with a score above 70?",

    # V8: Comparative Query (Subquery vs. specific value)
    "Find students with marks higher than Priya's",

    # V9: Time/Date Query (WHERE on a date column)
    "Which students were admitted after 2020",

    # V10: Nested Query (Subquery vs. aggregate)
    "Find students with marks greater than the average",

    # --- Bonus Test Cases (Extending the requirements) ---

    # Extends V3: Another aggregation (COUNT)
    "How many students are there?",

    # Extends V4: More complex sorting (ORDER BY with LIMIT)
    "Who is the oldest student?",

    # Extends V6: More complex JOIN (JOIN with a WHERE clause)
    "Show me the names and courses for students with grade A",

    # New Concept: Uniqueness (DISTINCT)
    "What are the distinct grades?",
]

print("\n --- Generating SQL Queries with Gemini --- \n")
for i, nlq in enumerate(queries_to_test):
    print(f"({i+1}) Natural Language:")
    print(f"    '{nlq}'")

    generated_sql = generate_sql_with_gemini(nlq, schema_statements)

    print(f"    ➡️ Generated SQL:")
    print(f"    {generated_sql}")
    print("-" * 50)



--- Database Schema Context ---

CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    grade TEXT,
    marks INTEGER,
    date_of_admission DATE
);
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL,
    student_id INTEGER,
    FOREIGN KEY (student_id) REFERENCES students (id)
);


 --- Generating SQL Queries with Gemini --- 

(1) Natural Language:
    'Show all students with marks > 80'
    ➡️ Generated SQL:
    SELECT *
FROM students
WHERE marks > 80;
--------------------------------------------------
(2) Natural Language:
    'List students older than 20 and with grade 'A''
    ➡️ Generated SQL:
    SELECT *
FROM students
WHERE age > 20 AND grade = 'A';
--------------------------------------------------
(3) Natural Language:
    'Find the average marks of students'
    ➡️ Generated SQL:
    SELECT AVG(marks) AS average_marks
FROM students;
--------------------------------------------------
(4) Natural L

# **Interactive Custom Schema Tester**

In [ ]:

# This cell allows you to provide a custom schema and then ask questions about it.

# First, check if the main generation function exists from the previous cell.
if 'generate_sql_with_gemini' not in globals():
    print("🛑 Error: The main project cell has not been run yet.")
    print("Please run the cell that defines the 'generate_sql_with_gemini' function first.")
else:
    # --- Step 1: Get the custom schema from the user ---
    print(" Please enter your database schema using CREATE TABLE statements.")
    print("  When you are finished, type 'done' on a new line and press Enter.")
    print("-" * 60)

    schema_lines = []
    while True:
        line = input()
        if line.lower().strip() == 'done':
            break
        schema_lines.append(line)

    custom_schema_statements = "\n".join(schema_lines)

    if not custom_schema_statements:
        print("\nNo schema provided. Exiting.")
    else:
        print("\n✅ Schema captured successfully!")
        print("Now, you can ask questions about your tables.")
        print("Type 'exit' or 'quit' to stop.")
        print("-" * 60)

        # --- Step 2: Loop for questions ---
        while True:
            nl_query = input("Your question: ")

            if nl_query.lower().strip() in ['exit', 'quit']:
                print("Exiting interactive session. Goodbye!")
                break

            if not nl_query.strip():
                continue

            # Generate the SQL using the custom schema
            generated_sql = generate_sql_with_gemini(nl_query, custom_schema_statements)

            print("\n    ➡️ Generated SQL:")
            print(f"    {generated_sql}\n")
            print("-" * 60)
